<a href="https://colab.research.google.com/github/abhranil-datascience/LSTM_POC/blob/master/MNISTClassificationPOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
############################## Mount Drive ######################################## 
#from google.colab import drive
#drive.mount('/content/gdrive')
############################## Change Directory ###################################
import os
os.chdir('/content/gdrive/My Drive/MLandDLFullCourse/DL/AdvancedNLP/2.BidirectionalRNN')
BestClassiferModel="BestClassiferModelLSTM.hdf5"
###################### Convert Digits to 28 x 28 matrix ###################
def CreateMNISTDigitsMatrix(DigitDataset):
  import numpy as np
  DigitDataset=DigitDataset/255.0
  DigitsImageMatrix=DigitDataset.reshape(42000,28,28)
  DigitsImageMatrixRotatedByNinetyDegree=np.transpose(DigitsImageMatrix,(0,2,1))
  return DigitsImageMatrix,DigitsImageMatrixRotatedByNinetyDegree
############################# Create Model #######################################
def CreateRNNModel(input_shape):
  from keras.layers import Input,Bidirectional,LSTM,Concatenate,Dense,GlobalMaxPooling1D,Dropout
  from keras.models import Model
  # Left To Right
  DigitImageLeftToRightInputData = Input(shape=input_shape)
  DigitImageLeftToRightLSTM = Bidirectional(LSTM(units=20,return_sequences=True))
  DigitImageLeftToRightLSTMHiddenUnits = DigitImageLeftToRightLSTM(DigitImageLeftToRightInputData)
  DigitImageLeftToRightLSTMHiddenUnitsMaxPooled=GlobalMaxPooling1D()(DigitImageLeftToRightLSTMHiddenUnits)
  # Top To Bottom
  DigitImageTopToBottomInputData = Input(shape=input_shape)
  DigitImageTopToBottomLSTM = Bidirectional(LSTM(units=20,return_sequences=True))
  DigitImageTopToBottomLSTMHiddenUnits = DigitImageTopToBottomLSTM(DigitImageTopToBottomInputData)
  DigitImageTopToBottomLSTMHiddenUnitsMaxPooled = GlobalMaxPooling1D()(DigitImageTopToBottomLSTMHiddenUnits)
  # Concatenate Both the Hidden Units
  concatenator=Concatenate(axis=1)
  ConcatenatedHiddenUnits=concatenator([DigitImageLeftToRightLSTMHiddenUnitsMaxPooled, DigitImageTopToBottomLSTMHiddenUnitsMaxPooled])
  # Apply Dense Layer
  ANNHiddenLayer1=Dense(units=256,activation='relu',kernel_initializer='random_uniform')(ConcatenatedHiddenUnits)
  X=Dropout(0.2)(ANNHiddenLayer1)
  ANNHiddenLayer2=Dense(units=256,activation='relu',kernel_initializer='random_uniform')(X)
  X=Dropout(0.2)(ANNHiddenLayer2)
  output=Dense(units=10,activation='softmax',kernel_initializer='random_uniform')(X)
  model=Model([DigitImageLeftToRightInputData,DigitImageTopToBottomInputData],output)
  model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
  return model
############################# Train Model #######################################
def TrainModel(DigitsDatasetLefttoRight,DigitsDatasetTopToBottom,labels,classifier,Epochs,BatchSize,ValidationSplit):
  from keras.callbacks import ModelCheckpoint
  SaveBestModel=ModelCheckpoint(filepath=BestClassiferModel,monitor='val_acc', verbose=1, save_best_only=True, mode='max')
  classifier.fit([DigitsDatasetLefttoRight,DigitsDatasetTopToBottom],labels,epochs=Epochs,batch_size=BatchSize,validation_split=ValidationSplit,callbacks=[SaveBestModel])
############################# Read Dataset #######################################
#!wget https://raw.githubusercontent.com/drsimonj/kaggle-digit-recognizer/master/data/train.csv
import pandas as pd
Dataset=pd.read_csv('train.csv')
#Dataset=Dataset.sample(frac=1).reset_index(drop=True)# Shuffles Dataset
Lables=Dataset.iloc[:,0].values
Digits=Dataset.iloc[:,1:].values
DigitsImageMatrix,DigitsImageMatrixRotatedByNinetyDegree=CreateMNISTDigitsMatrix(Digits)
Classifier=CreateRNNModel(DigitsImageMatrix[0].shape)
TrainModel(DigitsImageMatrix,DigitsImageMatrixRotatedByNinetyDegree,Lables,Classifier,30,128,0.2)
             




Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 33600 samples, validate on 8400 samples
Epoch 1/30
33600/33600 [==============================] - 56s 2ms/step - loss: 0.8204 - acc: 0.7307 - val_loss: 0.2929 - val_acc: 0.9107

Epoch 00001: val_acc improved from -inf to 0.91071, saving model to BestClassiferModelLSTM.hdf5
Epoch 2/30
33600/33600 [==============================] - 54s 2ms/step - loss: 0.2455 - acc: 0.9248 - val_loss: 0.1813 - val_acc: 0.9410

Epoch 00002: val_acc improved from 0.91071 to 0.94095, saving model to BestClassiferModelLSTM.hdf5
Epoch 3/30
33600/33600 [==============================] - 54s 2ms/step - loss: 0.1730 - acc: 0.9470 - val_loss: 0.1462 - val_acc: 0.9525

Epoch 00003: val_acc improved